# IDEAS

### Inspiring Kernels
* https://www.kaggle.com/harvinderwazir/match-outcome-prediction-in-football

* https://www.kaggle.com/stvrdo/goals-ratings-simulations/notebook

* https://www.kaggle.com/petehodge/epl-weekly-predicting/notebook
    
### VISUALIZATIONS - MUY IMPORTANTE
[Seventeen Ways to Map Data in Kaggle kernels](http://blog.kaggle.com/2016/11/30/seventeen-ways-to-map-data-in-kaggle-kernels/) 


## Background


# TYPE FROM PROPOSAL


## Data Preprocessing

### Type of data

The dataset is available at https://www.kaggle.com/hugomathien/soccer/data. The data is in the form of a sqlite database. 

### About the data

The data contains the following tables:

*** Country, League, Match, Player, Player_Attributes, Team, Team_Attributes***

The tables *country, league, player and team* are lookup tables. Lets focus on the table with attributes - Player_Attributes, Team_Attributes and Match.
In the following code cell, I read the SQLITE table and display the first record of all these three tables.

In [1]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from time import time
from IPython.display import display # Allows the use of display() for DataFrames
import data_preprocess as dp
# Import supplementary visualization code visuals.py, renamed as visuals_udacity.py. I have taken this code from the Udacity
# machine learning projects
from Utils import *

# Pretty display for notebooks
%matplotlib inline
conn = dp.uncompress_and_open_sqlite()
print ("Players")
players_df = dp.sql_to_dataframe(conn, select_all_query_table("Player_Attributes"))
display(players_df.head(n=1))
print ("Teams")
teams_df = dp.sql_to_dataframe(conn, select_all_query_table("Team_Attributes"))
display(teams_df.head(n=1))
print ("Matches")
match_df = dp.sql_to_dataframe(conn, select_all_query_table("Match"))
display(match_df.head(n=1))

Players


,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0


Teams


,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover


Matches


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.0,1.65,3.4,4.5,1.78,3.25,4.0,1.73,3.4,4.2


### Defining the data features and labels
# TODO: Put a small description of the features here.
* Players

The output label is the 'overall_rating' column; the features are the rest of columns (minus the metadata columns). Note that we might drop some features in subsequent sections, but lets keep them all for now. We will be using the output label for assessing a team's rating based on individual players' ratings.

In [17]:
players_ratings_label = players_df[['player_api_id', 'overall_rating', 'finishing', 'sliding_tackle', 'gk_reflexes', 'date']]
players_features_df = players_df.drop(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating', 'potential'], axis=1)
display(players_features_df.head(n=1))

,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0


* Teams
No output label. I'll drop the metadata to get features.

In [3]:
teams_features_df = teams_df.drop(['id', 'team_fifa_api_id', 'team_api_id', 'date'], axis=1)
display(teams_features_df.head(n=1))

,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,60,Balanced,NaN,Little,50,Mixed,Organised,60,Normal,65,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover


* Clustering matches based on excitement - This is an unsupervised learning problem, and excitement is kind of hard to describe. I will consider excitement in terms of the number of goals, red cards, incidents etc. See later section for implementation.

## TODO: Assign a player 'defense', 'attack' or 'midfield' based on the values of tackle, finishing etc. Validate against top players like Lionel messi, John Terry, Cristiano Ronaldo etc.

In [19]:
reload(dp)
player_names_df = dp.sql_to_dataframe(conn, select_all_query_table("Player"))
player_join_df = pd.merge(players_ratings_label, player_names_df, on='player_api_id', how='outer')
PLAYER_NAMES = ['Lionel Messi', 'Cristiano Ronaldo', 'Xavi Hernandez', 'John Terry', 'Petr Cech', 'Frank Lampard', 'Didier Drogba',
               'Zlatan Ibrahimovic', 'Wayne Rooney', 'Karim Benzema']
for player in PLAYER_NAMES:
    display(dp.players_to_work_rate(players_ratings_label=players_ratings_label, conn=conn, player_name=player))

,player_name,finishing,sliding_tackle,gk_reflexes
102482,Lionel Messi,93.0,21.0,8.0


,player_name,finishing,sliding_tackle,gk_reflexes
33330,Cristiano Ronaldo,95.0,23.0,11.0


,player_name,finishing,sliding_tackle,gk_reflexes
180709,Xavi Hernandez,74.0,59.0,9.0


,player_name,finishing,sliding_tackle,gk_reflexes
84848,John Terry,46.0,84.0,8.0


,player_name,finishing,sliding_tackle,gk_reflexes
142868,Petr Cech,12.0,12.0,85.0


,player_name,finishing,sliding_tackle,gk_reflexes
59244,Frank Lampard,72.0,57.0,8.0


,player_name,finishing,sliding_tackle,gk_reflexes
43418,Didier Drogba,83.0,29.0,14.0


,player_name,finishing,sliding_tackle,gk_reflexes
183672,Zlatan Ibrahimovic,90.0,27.0,12.0


,player_name,finishing,sliding_tackle,gk_reflexes
178940,Wayne Rooney,85.0,37.0,7.0


,player_name,finishing,sliding_tackle,gk_reflexes
93533,Karim Benzema,88.0,12.0,7.0


### Defining the base case: The home advantage
Home advantage is real, as mentioned in the Kaggle link, as well as here: http://bleacherreport.com/articles/1604854-how-much-does-home-field-advantage-matter-in-soccer. Lets analyze the home advantage across different leagues.


# ONE HOT ENCODE THE RESULT VECTOR

In [5]:
# TODO: Finish this function
home_advantage_per_country = home_advantage(match_df)

NameError: name 'home_advantage' is not defined


# DRAW VISUALIZATIONS TO SEE SOME DATA

Ref for visualizing data
# https://www.kaggle.com/stvrdo/goals-ratings-simulations/notebook
The data is in the zipfile 'soccer.zip', and is in the form of a sqlite database.

#### Defining the problems

* The holy grail is predicting the outcome of a football match, as mentioned in the kaggle link for the dataset I intend to use 

* Cluster matches based on the excitement level and interest in the game; possibly use this to model ticket prices. I would guess that a match between Real Madrid and Barcelona would be the most expensive. What factors make fans pay more for a match? If I follow a certain team, how much money can I expect to shell out?

* Model players and teams; and assign them a quantifiable number that serves as a rating of skill. I would like to find the answer to the pull of star power; many people tune in to football matches to watch Messi, Neymar or Ronaldo. Note that the capability of players also affects the excitement surrounding football matches.

* The main service they provided is advice to teams on whether or not a player should be bought or sold, not sure if this would be of any help, one of they key concepts they discussed was coming up with a metric on players on how much of an impact the player makes which was calculated by figuring out a team's performance with and without that particular player while taking into account other players and different circumstances.
**Background**
Players get injured all the time, with managers complaning about injuries (insert link). Through this part of the code, I wish to answer this queation: ***How does the team fare if player X is out due to injury for T% of the season?***. For eg. : How does Barcelona do in season 2011-12, had Messi not been there?

### Handling Bad Data
# HANDLE NaN VALUES
# USE DROP NA

## Feature Selection

#### Divide into Forward-Midfield-Defense for features and player rating

## Model Fitting and Optimization

### You Do not have the label for match excitement-therefore Clustering!

### Though it may sound high for such a random sport game, you've got to know that the home team wins about 46% of the time. So the base case (constantly predicting Home Win) has indeed 46% precision.

## Prediction Results and Optimizations

## Conclusion